### Задание 1

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

__Комментарий к решению__

Кажется, вышло два варианта: 1) просто функция, которая возвращает максимальный курс и название соответствующей валюты 2) отдельный класс, который работает с исходным классом Rate из лекции (навернео, можно было и в него вписать, но не стал портить новым кодом исходный Rate)

In [6]:
import requests

In [31]:
def main_name():
    
    r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
    temp_rate = 0
    dict_r = r.json()['Valute']

    for i in dict_r.values():
        
        x = float(i['Value'])/int(i['Nominal'])

        if  x > temp_rate and i['Value'] != dict_r['XDR']['Value']:

            temp_rate = x 
            temp_name = i['Name']

    return temp_rate, temp_name

In [32]:
main_name()

(87.5972, 'Фунт стерлингов Соединенного королевства')

In [14]:
class Rate:
    
    def __init__(self, format_='value'):
        self.format = format_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [34]:
class Names(Rate):
    
    def __init__(self):

        super().__init__(self)
        
    def super_rate(self):
        
        temp_name = ''
        temp_rate = 0

        for i in self.exchange_rates().values():
            
            x = float(i['Value']) / int(i['Nominal'])

            if x > temp_rate and i['Value'] != self.exchange_rates()['XDR']['Value']:

                temp_rate = x
                temp_name = i['Name']

        return temp_rate, temp_name

In [35]:
f = Names()

In [36]:
f.super_rate()

(87.5972, 'Фунт стерлингов Соединенного королевства')

### Задание 2

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

__Комментарий к решению__

Не придумал лучше, чем засунуть self.diff внутрь метода eur или usd — чтобы передавать сразу, как запрашиваем, что хотим получить, курс или разницу

In [124]:
class Rate:
    
    def __init__(self, format_='value'):
        self.format = format_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self, diff):
        """Возвращает курс евро на сегодня в формате self.format — либо разницу с прошлым если передан diff равные True"""
        
        self.diff = diff
        
        if self.diff == True:
            return float(self.exchange_rates()['EUR']['Previous'] - self.exchange_rates()['EUR']['Value'])
        
        else:
              return self.make_format('EUR')
    
    def usd(self, diff):
        """Возвращает курс доллара на сегодня в формате self.format либо разницу с предыдущим курсом"""
      
        self.diff = diff
        if self.diff == True:
            return float(self.exchange_rates()['USD']['Previous'] - self.exchange_rates()['USD']['Value'])
        
        else:
            return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [125]:
xe = Rate()

In [126]:
xe.eur(diff=True)

-2.1581999999999937

In [127]:
xe.eur(diff=False)

75.8424

In [128]:
xe.usd(diff=True)

-1.332099999999997

In [129]:
xe.usd(diff=False)

67.5175

# Домашнее задание задача 3

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

Для проверки используйте код:

```python
elena = Designer('Елена', seniority=0, awards=2)
```

In [130]:
class Employee:
    
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

In [131]:
class Designer(Employee):
    
    def __init__ (self, name, seniority, awards):
        
        super().__init__(name, seniority)
        
        self.awards = awards
        self.seniority = seniority + self.awards * 2
        
    def check_if_it_is_time_for_upgrade(self):
        """Проводит аккредитацию, добавляя балл за каждую, и проверяет,\
        не пора ли повысить грейд"""
        
        self.seniority += 1
        
        if self.seniority % 7 == 0:
            
            self.grade_up()
            
        return self.publish_grade()

In [132]:
elena = Designer('Елена', seniority=0, awards=2)

In [133]:
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
Елена 4
Елена 4
